In [1]:
import pandas as pd

# Set the options to display all the columns
pd.set_option('display.max_colwidth', None)

In [2]:
!pip install selenium


In [3]:
!pip install webdriver_manager

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [13]:
!apt update
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb
!apt --fix-broken install -y
!rm chrome.deb  # 설치 후 삭제


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
31 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [16]:
!google-chrome --version


Google Chrome 133.0.6943.141 


In [20]:
!CHROME_VERSION=$(google-chrome --version | awk '{print $3}' | cut -d. -f1)
!DRIVER_VERSION=$(curl -s "https://chromedriver.storage.googleapis.com/LATEST_RELEASE_$CHROME_VERSION")
!wget -q -O chromedriver.zip "https://chromedriver.storage.googleapis.com/$DRIVER_VERSION/chromedriver_linux64.zip"
!unzip chromedriver.zip
!chmod +x chromedriver
!mv chromedriver /usr/bin/chromedriver
!rm chromedriver.zip



Archive:  chromedriver.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of chromedriver.zip or
        chromedriver.zip.zip, and cannot find chromedriver.zip.ZIP, period.
chmod: cannot access 'chromedriver': No such file or directory
mv: cannot stat 'chromedriver': No such file or directory


In [25]:
!chromedriver --version



Command '/usr/bin/chromedriver' requires the chromium snap to be installed.
Please install it with:

snap install chromium



In [32]:
# Set web driver
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5) # Wait until the action happens

data = []

while True:
    # Open up the page
    url = 'https://www.kkday.com/ko/category/global/experiences/list?prodcat=CATEGORY_025,CATEGORY_020,CATEGORY_022,CATEGORY_023,CATEGORY_030,CATEGORY_028,CATEGORY_024,CATEGORY_029,CATEGORY_026,CATEGORY_021,CATEGORY_027,CATEGORY_046,CATEGORY_044,CATEGORY_045,CATEGORY_033,CATEGORY_037,CATEGORY_032,CATEGORY_034,CATEGORY_038,CATEGORY_036,CATEGORY_041,CATEGORY_040,CATEGORY_039,CATEGORY_042,CATEGORY_094,CATEGORY_035,CATEGORY_050,CATEGORY_048,CATEGORY_049,CATEGORY_051,CATEGORY_053,CATEGORY_054&currency=KRW&sort=prec&count=10&page=1&row=10&immediately_use=0&sale_status=0'
    driver.get(url)

    # if there is a pop-up regarding cookies, close it
    try:
        close_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@id="CybotCookiebotDialogBodyButtonAccept"]')))
        close_button.click()
    except:
        print("No cookie pop-up.")

    # Find each product cards
    products = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-listview search-info"]')))
    for product in products:
        product_name = product.find_element(By.XPATH, './/span[@class="product-listview__name"]').text
        description = product.find_element(By.XPATH, './/p[@class="description"]').text
        region = product.find_element(By.XPATH, './/div[@class="product-listview__inco_info--flex_item product-place"]').text

        product_star = wait.until(EC.presence_of_element_located((By.XPATH, './/div[@class="product-star"]')))

        # 모든 별(svg) 요소 찾기
        stars = product_star.find_elements(By.XPATH, './/svg[contains(@class, "custom-kk-icon star")]')

        # 평점 계산
        rating = 0.0
        for star in stars:
            star_class = star.get_attribute("class")  # class 속성 가져오기
            if "star-100" in star_class:
                rating += 1.0
            elif "star-75" in star_class:
                rating += 0.75
            elif "star-50" in star_class:
                rating += 0.5
            elif "star-25" in star_class:
                rating += 0.25

        number_of_reviews = product_star.find_element(By.XPATH, './/span[@class="text"]').text
        number_of_bookings = product.find_element(By.XPATH, './/div[@class="product-book"]').text
        price = product.find_element(By.XPATH, './/div[@class="kk-price-local__normal"]').text
        data.append([product_name, description, region, rating, number_of_reviews, number_of_bookings, price])
    break

data
#     driver.execute_script("arguments[0].click();", city_select_button)

#     # Find the city list container (without clicking)
#     city_list = city_select_button.find_element(By.XPATH, "following-sibling::div//ul")

#     # Extract all <a> elements (cities) inside the <ul>
#     cities = city_list.find_elements(By.CSS_SELECTOR, "a.LocationCityList-module__link--MEsOD")

#     # Extract city names and links
#     for city in cities:
#         city_name = driver.execute_script("return arguments[0].textContent.trim();", city)
#         city_link = city.get_attribute("href")
#         data.append([country, city_name, city_link])

# driver.quit()

# # Convert to DataFrame
# cities_link = pd.DataFrame(data, columns=['Country', 'City', 'Link'])

# # Display the DataFrame to verify the data
# print(cities_link)

# # Save the DataFrame as a CSV file
# # cities_link.to_csv('cities_link.csv', index=False)

[['[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)',
  '지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.',
  '카와구치호',
  0.0,
  '(7,039)',
  '30K+ 예약',
  '44,294'],
 ['타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인)',
  '지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!',
  '타이중',
  0.0,
  '(7,039)',
  '9K+ 예약',
  '26,191'],
 ['구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발)',
  '지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.',
  '켄로쿠엔',
  0.0,
  '(7,039)',
  '1K+ 예약',
  '620,597'],
 ['다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능)',
  '지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!',
  '다낭',
  0.0,
  '(7,039)',
  '5K+ 예약',
  '5,050'],
 ['[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인)',
  '출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!',
  '르네이 향 (仁愛鄉)',
  0.0,
  '(7,039)',
  '8K+ 예약',
  

In [29]:
pd.DataFrame(data, columns=['Product Name', 'Description', 'Region', 'Rating', 'Number of Reviews', 'Number of Bookings', 'Price'])

,Product Name,Description,Region,Rating,Number of Reviews,Number of Bookings,Price
0,"[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)","지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.",카와구치호,0.0,"(7,039)",30K+ 예약,"44,294"
1,타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인),"지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!",타이중,0.0,"(7,039)",9K+ 예약,"26,191"
2,구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발),지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.,켄로쿠엔,0.0,"(7,039)",1K+ 예약,"620,597"
3,다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능),"지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!",다낭,0.0,"(7,039)",5K+ 예약,"5,050"
4,[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인),"출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!",르네이 향 (仁愛鄉),0.0,"(7,039)",8K+ 예약,"48,098"
5,자이 아리산 일일 투어 | 펀치후 라오제 & 아리산 (타이중 시내 출발),"지금 바로 KKday에서 자이 아리산 조인 일일 투어를 예약하고 타이중에서 출발해 펀치후 라오제, 아리산으로 떠나보세요!",아리산,0.0,"(7,039)",2K+ 예약,"63,402"
6,도쿄 닛코 일일 투어|닛코 도쇼구 & 게곤 폭포 & 닛코온천 & 닛코 에도 마을 (긴자 출발),"지금 바로 KKday에서 합리적인 가격으로 도쿄 닛코 일일 투어를 예약하고, 닛코 도쇼구, 주젠지호, 게곤 폭포, 닛코 온천 및 닛코 에도 마을 등 필수 방문 명소를 둘러보세요!",계곤폭포 (Gyegon pokpo),0.0,"(7,039)",7K+ 예약,"70,171"
7,필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 모알보알 호핑 일일 투어,"지금 바로 KKday에서 합리적인 가격으로 필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 섬호핑 일일 투어를 예약하고, {고래상어들을 가까이서 관찰하고, 투말록 폭포를 감상하며 모알보알 페스카도르에서 섬 호핑을 즐겨보세요!",세부,0.0,"(7,039)",2K+ 예약,"21,051"
8,방콕 아유타야 일일 투어 | 아유타야 사원 & 방파인 궁전 & 딸랏 꿍 & 카페 (한국어 가이드),"지금 바로 KKday에서 합리적인 가격으로 방콕 아유타야 일일 투어를 예약하고, 태국의 역사가 숨쉬는 불멸의 도시, 아유타야로 떠나 유네스코 문화유산과 경이로운 유적지를 만나보세요! Ayudhaya ayudhya",방콕,0.0,"(7,039)",4K+ 예약,"41,200"
9,대만 예스폭지 일일 투어 | 지우펀 & 예류 & 스펀 & 아메이차루 & 황금폭포 (고속철도 승차권 할인),"지금 KKday에서 할인 가격으로 대만 타이베이 근교 일일 투어을 예약하고, 지우펀, 예류, 스펀 핑시, 아메이차루를 방문하세요!",핑시,0.0,"(7,039)",40K+ 예약,"30,302"


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Selenium 설정
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)

df = []

# 페이지 열기
url = 'https://www.kkday.com/ko/category/global/experiences/list?prodcat=CATEGORY_025,CATEGORY_020,CATEGORY_022,CATEGORY_023,CATEGORY_030,CATEGORY_028,CATEGORY_024,CATEGORY_029,CATEGORY_026,CATEGORY_021,CATEGORY_027,CATEGORY_046,CATEGORY_044,CATEGORY_045,CATEGORY_033,CATEGORY_037,CATEGORY_032,CATEGORY_034,CATEGORY_038,CATEGORY_036,CATEGORY_041,CATEGORY_040,CATEGORY_039,CATEGORY_042,CATEGORY_094,CATEGORY_035,CATEGORY_050,CATEGORY_048,CATEGORY_049,CATEGORY_051,CATEGORY_053,CATEGORY_054&currency=KRW&sort=prec&count=10&page=1&row=10&immediately_use=0&sale_status=0'
driver.get(url)

# 쿠키 팝업 닫기
try:
    close_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@id="CybotCookiebotDialogBodyButtonAccept"]')))
    close_button.click()
except:
    print("No cookie pop-up.")

# 상품 리스트 찾기
products = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-listview search-info"]')))

for product in products:
    product_name = product.find_element(By.XPATH, './/span[@class="product-listview__name"]').text
    description = product.find_element(By.XPATH, './/p[@class="description"]').text
    region = product.find_element(By.XPATH, './/div[@class="product-listview__inco_info--flex_item product-place"]').text

    # 별점 정보 찾기
    try:
        product_star = product.find_element(By.XPATH, './/div[contains(@class, "product-star")]')
        stars = product_star.find_elements(By.XPATH, './/svg[contains(@class, "custom-kk-icon star")]')

        rating = 0.0
        for star in stars:
            star_class = star.get_attribute("class")
            if "star-100" in star_class:
                rating += 1.0
            elif "star-75" in star_class:
                rating += 0.75
            elif "star-50" in star_class:
                rating += 0.5
            elif "star-25" in star_class:
                rating += 0.25
    except:
        rating = 0.0

    # 리뷰 수 찾기
    try:
        number_of_reviews = product_star.find_element(By.XPATH, './/span[contains(@class, "text")]').text
    except:
        number_of_reviews = "0"

    # 예약 수 찾기
    try:
        number_of_bookings = product.find_element(By.XPATH, './/div[@class="product-book"]').text
    except:
        number_of_bookings = "N/A"

    # 가격 찾기
    try:
        price = product.find_element(By.XPATH, './/div[@class="kk-price-local__normal"]').text
    except:
        price = "N/A"

    df.append([product_name, description, region, rating, number_of_reviews, number_of_bookings, price])

driver.quit()

# 결과 확인
df


[['[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)',
  '지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.',
  '카와구치호',
  0.0,
  '(7,039)',
  '30K+ 예약',
  '44,294'],
 ['타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인)',
  '지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!',
  '타이중',
  0.0,
  '(1,945)',
  '9K+ 예약',
  '26,191'],
 ['구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발)',
  '지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.',
  '켄로쿠엔',
  0.0,
  '(190)',
  '1K+ 예약',
  '620,597'],
 ['다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능)',
  '지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!',
  '다낭',
  0.0,
  '(437)',
  '5K+ 예약',
  '5,050'],
 ['[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인)',
  '출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!',
  '르네이 향 (仁愛鄉)',
  0.0,
  '(2,539)',
  '8K+ 예약',
  '48,

In [34]:
pd.DataFrame(df, columns=['Product Name', 'Description', 'Region', 'Rating', 'Number of Reviews', 'Number of Bookings', 'Price'])

,Product Name,Description,Region,Rating,Number of Reviews,Number of Bookings,Price
0,"[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)","지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.",카와구치호,0.0,"(7,039)",30K+ 예약,"44,294"
1,타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인),"지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!",타이중,0.0,"(1,945)",9K+ 예약,"26,191"
2,구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발),지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.,켄로쿠엔,0.0,(190),1K+ 예약,"620,597"
3,다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능),"지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!",다낭,0.0,(437),5K+ 예약,"5,050"
4,[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인),"출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!",르네이 향 (仁愛鄉),0.0,"(2,539)",8K+ 예약,"48,098"
5,자이 아리산 일일 투어 | 펀치후 라오제 & 아리산 (타이중 시내 출발),"지금 바로 KKday에서 자이 아리산 조인 일일 투어를 예약하고 타이중에서 출발해 펀치후 라오제, 아리산으로 떠나보세요!",아리산,0.0,(728),2K+ 예약,"63,402"
6,도쿄 닛코 일일 투어|닛코 도쇼구 & 게곤 폭포 & 닛코온천 & 닛코 에도 마을 (긴자 출발),"지금 바로 KKday에서 합리적인 가격으로 도쿄 닛코 일일 투어를 예약하고, 닛코 도쇼구, 주젠지호, 게곤 폭포, 닛코 온천 및 닛코 에도 마을 등 필수 방문 명소를 둘러보세요!",계곤폭포 (Gyegon pokpo),0.0,"(1,849)",7K+ 예약,"70,171"
7,필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 모알보알 호핑 일일 투어,"지금 바로 KKday에서 합리적인 가격으로 필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 섬호핑 일일 투어를 예약하고, {고래상어들을 가까이서 관찰하고, 투말록 폭포를 감상하며 모알보알 페스카도르에서 섬 호핑을 즐겨보세요!",세부,0.0,(553),2K+ 예약,"21,051"
8,방콕 아유타야 일일 투어 | 아유타야 사원 & 방파인 궁전 & 딸랏 꿍 & 카페 (한국어 가이드),"지금 바로 KKday에서 합리적인 가격으로 방콕 아유타야 일일 투어를 예약하고, 태국의 역사가 숨쉬는 불멸의 도시, 아유타야로 떠나 유네스코 문화유산과 경이로운 유적지를 만나보세요! Ayudhaya ayudhya",방콕,0.0,"(1,147)",4K+ 예약,"41,200"
9,대만 예스폭지 일일 투어 | 지우펀 & 예류 & 스펀 & 아메이차루 & 황금폭포 (고속철도 승차권 할인),"지금 KKday에서 할인 가격으로 대만 타이베이 근교 일일 투어을 예약하고, 지우펀, 예류, 스펀 핑시, 아메이차루를 방문하세요!",핑시,0.0,"(12,994)",40K+ 예약,"30,302"


In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Selenium 설정
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)

df1 = []

# 페이지 열기
url = 'https://www.kkday.com/ko/category/global/experiences/list?prodcat=CATEGORY_025,CATEGORY_020,CATEGORY_022,CATEGORY_023,CATEGORY_030,CATEGORY_028,CATEGORY_024,CATEGORY_029,CATEGORY_026,CATEGORY_021,CATEGORY_027,CATEGORY_046,CATEGORY_044,CATEGORY_045,CATEGORY_033,CATEGORY_037,CATEGORY_032,CATEGORY_034,CATEGORY_038,CATEGORY_036,CATEGORY_041,CATEGORY_040,CATEGORY_039,CATEGORY_042,CATEGORY_094,CATEGORY_035,CATEGORY_050,CATEGORY_048,CATEGORY_049,CATEGORY_051,CATEGORY_053,CATEGORY_054&currency=KRW&sort=prec&count=10&page=1&row=10&immediately_use=0&sale_status=0'
driver.get(url)

# 쿠키 팝업 닫기
try:
    close_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@id="CybotCookiebotDialogBodyButtonAccept"]')))
    close_button.click()
except:
    print("No cookie pop-up.")

# 상품 리스트 찾기
products = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-listview search-info"]')))

for product in products:
    product_name = product.find_element(By.XPATH, './/span[@class="product-listview__name"]').text
    description = product.find_element(By.XPATH, './/p[@class="description"]').text
    region = product.find_element(By.XPATH, './/div[@class="product-listview__inco_info--flex_item product-place"]').text

    # ⭐ 별점 정보 찾기
    try:
        product_star = product.find_element(By.XPATH, './/div[contains(@class, "product-star")]')

        # 별표(svg) 로딩을 기다림
        stars = WebDriverWait(product_star, 3).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "svg"))
        )

        rating = 0.0
        print(f"🔹 상품: {product_name}")  # 상품명 출력
        print(f"🔹 별 개수: {len(stars)}")  # 별 개수 출력

        for star in stars:
            star_html = star.get_attribute("outerHTML")  # HTML 코드 가져오기
            print(f"🔹 별 HTML: {star_html}")  # 별 태그 HTML 출력

            if 'star-100' in star_html:
                rating += 1.0
            elif 'star-75' in star_html:
                rating += 0.75
            elif 'star-50' in star_html:
                rating += 0.5
            elif 'star-25' in star_html:
                rating += 0.25
    except Exception as e:
        print(f"🚨 별점 정보 오류: {e}")
        rating = 0.0

    # 리뷰 수 찾기
    try:
        number_of_reviews = product_star.find_element(By.XPATH, './/span[contains(@class, "text")]').text
    except:
        number_of_reviews = "0"

    # 예약 수 찾기
    try:
        number_of_bookings = product.find_element(By.XPATH, './/div[@class="product-book"]').text
    except:
        number_of_bookings = "N/A"

    # 가격 찾기
    try:
        price = product.find_element(By.XPATH, './/div[@class="kk-price-local__normal"]').text
    except:
        price = "N/A"

    df1.append([product_name, description, region, rating, number_of_reviews, number_of_bookings, price])

driver.quit()

# 결과 확인
df1


🔹 상품: [1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)
🔹 별 개수: 5
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-75"><use xlink:href="#ic_star"></use></svg>
🔹 상품: 타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인)
🔹 별 개수: 5
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-100"><use xlink:href="#ic_star"></use></svg>
🔹 별 HTML: <svg class="custom-kk-icon star star-75"

[['[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)',
  '지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.',
  '카와구치호',
  4.75,
  '(7,039)',
  '30K+ 예약',
  '44,294'],
 ['타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인)',
  '지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!',
  '타이중',
  4.75,
  '(1,945)',
  '9K+ 예약',
  '26,191'],
 ['구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발)',
  '지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.',
  '켄로쿠엔',
  4.5,
  '(190)',
  '1K+ 예약',
  '620,597'],
 ['다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능)',
  '지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!',
  '다낭',
  4.5,
  '(437)',
  '5K+ 예약',
  '5,050'],
 ['[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인)',
  '출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!',
  '르네이 향 (仁愛鄉)',
  4.75,
  '(2,539)',
  '8K+ 예약',
  '

아래 코드가 최종적으로 해결 완료한 코드입니다

In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Selenium 설정
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)

df2 = []

# 페이지 열기
url = 'https://www.kkday.com/ko/category/global/experiences/list?prodcat=CATEGORY_025,CATEGORY_020,CATEGORY_022,CATEGORY_023,CATEGORY_030,CATEGORY_028,CATEGORY_024,CATEGORY_029,CATEGORY_026,CATEGORY_021,CATEGORY_027,CATEGORY_046,CATEGORY_044,CATEGORY_045,CATEGORY_033,CATEGORY_037,CATEGORY_032,CATEGORY_034,CATEGORY_038,CATEGORY_036,CATEGORY_041,CATEGORY_040,CATEGORY_039,CATEGORY_042,CATEGORY_094,CATEGORY_035,CATEGORY_050,CATEGORY_048,CATEGORY_049,CATEGORY_051,CATEGORY_053,CATEGORY_054&currency=KRW&sort=prec&count=10&page=1&row=10&immediately_use=0&sale_status=0'
driver.get(url)

# 쿠키 팝업 닫기
try:
    close_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@id="CybotCookiebotDialogBodyButtonAccept"]')))
    close_button.click()
except:
    pass

# 상품 리스트 찾기
products = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-listview search-info"]')))

for product in products:
    product_name = product.find_element(By.XPATH, './/span[@class="product-listview__name"]').text
    description = product.find_element(By.XPATH, './/p[@class="description"]').text
    region = product.find_element(By.XPATH, './/div[@class="product-listview__inco_info--flex_item product-place"]').text

    # ⭐ 별점 정보 찾기
    try:
        product_star = product.find_element(By.XPATH, './/div[contains(@class, "product-star")]')

        # 별표(svg) 로딩을 기다림
        stars = WebDriverWait(product_star, 3).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "svg"))
        )

        rating = 0.0
        for star in stars:
            star_html = star.get_attribute("outerHTML")  # HTML 코드 가져오기

            if 'star-100' in star_html:
                rating += 1.0
            elif 'star-75' in star_html:
                rating += 0.75
            elif 'star-50' in star_html:
                rating += 0.5
            elif 'star-25' in star_html:
                rating += 0.25
    except:
        rating = 0.0

    # 리뷰 수 찾기
    try:
        number_of_reviews = product_star.find_element(By.XPATH, './/span[contains(@class, "text")]').text
    except:
        number_of_reviews = "0"

    # 예약 수 찾기
    try:
        number_of_bookings = product.find_element(By.XPATH, './/div[@class="product-book"]').text
    except:
        number_of_bookings = "N/A"

    # 가격 찾기
    try:
        price = product.find_element(By.XPATH, './/div[@class="kk-price-local__normal"]').text
    except:
        price = "N/A"

    df2.append([product_name, description, region, rating, number_of_reviews, number_of_bookings, price])

driver.quit()

# 결과 출력
df2


[['[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)',
  '지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.',
  '카와구치호',
  4.75,
  '(7,039)',
  '30K+ 예약',
  '44,294'],
 ['타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인)',
  '지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!',
  '타이중',
  4.75,
  '(1,945)',
  '9K+ 예약',
  '26,191'],
 ['구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발)',
  '지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.',
  '켄로쿠엔',
  4.5,
  '(190)',
  '1K+ 예약',
  '620,597'],
 ['다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능)',
  '지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!',
  '다낭',
  4.5,
  '(437)',
  '5K+ 예약',
  '5,050'],
 ['[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인)',
  '출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!',
  '르네이 향 (仁愛鄉)',
  4.75,
  '(2,539)',
  '8K+ 예약',
  '

In [51]:
pd.DataFrame(df2, columns=['Product Name', 'Description', 'Region', 'Rating', 'Number of Reviews', 'Number of Bookings', 'Price'])

,Product Name,Description,Region,Rating,Number of Reviews,Number of Bookings,Price
0,"[1인출발] 일본 후지산 인기 명소 당일치기 투어 (도쿄 출발, 한국어가이드)","지금 KKday에서 후지산 당일치기 투어를 예약하세요. 도쿄에서 출발해 가와구치코, 오시노핫카이, 고텐바 프리미엄 아울렛 등을 방문하실 수 있답니다.",카와구치호,4.75,"(7,039)",30K+ 예약,"44,294"
1,타이중 반나절 투어 | 무지개마을 & 궁원안과 & 고미습지 (고속철도 승차권 할인),"지금 바로 KKday에서 할인 가격으로 타이중 고미습지 반나절 투어를 예약하고, 반나절 동안 교통 걱정없이 고미 습지를 다녀오며 아름다운 일몰을 눈에 담고 사진으로 남겨보세요!",타이중,4.75,"(1,945)",9K+ 예약,"26,191"
2,구로베 다테야마 눈벽 2박 3일 투어 (나고야 출발),지금 바로 KKday에서 할인 가격으로 구로베 다테야마 눈벽 2박 3일 투어를 예약하세요! 나고야에서 출발해 다테야마 구로베 눈의 대계곡의 아름다운 풍경을 쉽게 경험해보세요.,켄로쿠엔,4.50,(190),1K+ 예약,"620,597"
3,다낭 썬월드 바나힐 투어(왕복 차량+입장권(케이블카)/자유 일정/1인 출발 가능),"지금 KKday에서 썬월드 바나힐 일일 투어를 예약하고, 썬월드 바나힐의 시그니처 투어를 통해 전에 없던 특별한 모험을 떠나보세요!",다낭,4.50,(437),5K+ 예약,"5,050"
4,[출발 보장] 타이중 일월담 & 칭징농장 일일 투어 (유람선 티켓 포함)(고속철도 승차권 할인),"출발 보장! KKday에서 할인된 가격으로 타이중 일월담 & 칭징농장 시그니처 일일 투어를 예약하고, 편리하게 타이중 유명 명소를 관광하세요. 일월담 차예단(阿婆茶葉蛋) 단독 증정 중!",르네이 향 (仁愛鄉),4.75,"(2,539)",8K+ 예약,"48,098"
5,자이 아리산 일일 투어 | 펀치후 라오제 & 아리산 (타이중 시내 출발),"지금 바로 KKday에서 자이 아리산 조인 일일 투어를 예약하고 타이중에서 출발해 펀치후 라오제, 아리산으로 떠나보세요!",아리산,4.75,(728),2K+ 예약,"63,402"
6,도쿄 닛코 일일 투어|닛코 도쇼구 & 게곤 폭포 & 닛코온천 & 닛코 에도 마을 (긴자 출발),"지금 바로 KKday에서 합리적인 가격으로 도쿄 닛코 일일 투어를 예약하고, 닛코 도쇼구, 주젠지호, 게곤 폭포, 닛코 온천 및 닛코 에도 마을 등 필수 방문 명소를 둘러보세요!",계곤폭포 (Gyegon pokpo),4.50,"(1,849)",7K+ 예약,"70,171"
7,필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 모알보알 호핑 일일 투어,"지금 바로 KKday에서 합리적인 가격으로 필리핀 세부 고래상어 스노클링 & 투말록 폭포 & 섬호핑 일일 투어를 예약하고, {고래상어들을 가까이서 관찰하고, 투말록 폭포를 감상하며 모알보알 페스카도르에서 섬 호핑을 즐겨보세요!",세부,4.50,(553),2K+ 예약,"21,051"
8,방콕 아유타야 일일 투어 | 아유타야 사원 & 방파인 궁전 & 딸랏 꿍 & 카페 (한국어 가이드),"지금 바로 KKday에서 합리적인 가격으로 방콕 아유타야 일일 투어를 예약하고, 태국의 역사가 숨쉬는 불멸의 도시, 아유타야로 떠나 유네스코 문화유산과 경이로운 유적지를 만나보세요! Ayudhaya ayudhya",방콕,4.75,"(1,147)",4K+ 예약,"41,200"
9,대만 예스폭지 일일 투어 | 지우펀 & 예류 & 스펀 & 아메이차루 & 황금폭포 (고속철도 승차권 할인),"지금 KKday에서 할인 가격으로 대만 타이베이 근교 일일 투어을 예약하고, 지우펀, 예류, 스펀 핑시, 아메이차루를 방문하세요!",핑시,4.75,"(12,994)",40K+ 예약,"30,302"
